In [ ]:
# Step 1: Import Libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Load and Explore Data
df = pd.read_excel("/content/Telco_customer_churn 2.xlsx")

# Step 3: Preprocess Data
relevant_columns = ['Tenure Months', 'Tech Support', 'Churn Value']
df = df[relevant_columns]

# Convert 'Tech Support' to binary (0 for 'Yes', 1 for 'No')
df['Tech Support'] = df['Tech Support'].apply(lambda x: 0 if x.lower() == 'yes' else 1)

# Drop rows with missing values
df = df.dropna()

features = df[['Tenure Months', 'Tech Support']]
target = df['Churn Value']

# Step 4: Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Step 5: Hyperparameter Tuning using RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
random_search.fit(X_train, y_train)

print("Best Hyperparameters:", random_search.best_params_)

# Step 6: Train a Model with Best Hyperparameters
best_model = RandomForestClassifier(random_state=42, **random_search.best_params_)
best_model.fit(X_train, y_train)

# Step 7: Evaluate Model on Test Set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# Step 8: User Input for Prediction
tenure_months = float(input("Enter the customer's tenure months: "))
tech_support = input("Did the customer use tech support? (Yes/No): ").lower()
state = input("Enter the customer's state: ")

tech_support_encoded = 0 if tech_support == 'yes' else 1

# Make Prediction
user_prediction = best_model.predict([[tenure_months, tech_support_encoded]])

# Display Results
print("\nPrediction Results:")
print(f"State: {state}")
print(f"Tenure Months: {tenure_months}")
print(f"Used Tech Support: {tech_support}")
print(f"Prediction: {'Churn' if user_prediction[0] == 1 else 'Not Churn'}")


Best Hyperparameters: {'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Accuracy on the test set: 73.39%
